In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text \
import text_to_word_sequence
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
EPOCHS = 50
BATCH_SIZE = 8
WINDOW_LENGTH = 40
WINDOW_STEP = 3
PREDICT_LENGTH = 3
MAX_WORDS = 10000
EMBEDDING_WIDTH = 100

In [ ]:
# Open and read file.
INPUT_FILE_NAME = 'Frankenstein.txt'
file = open(INPUT_FILE_NAME, 'r', encoding='utf-8')
text = file.read()
file.close()

In [ ]:
# Make lower case and split into individual words.
text = text_to_word_sequence(text)

In [ ]:
print(text)

In [ ]:
# Create training examples.
fragments = []
targets = []
for i in range(0, len(text) - WINDOW_LENGTH, WINDOW_STEP):
    fragments.append(text[i: i + WINDOW_LENGTH])
    targets.append(text[i + WINDOW_LENGTH])

In [ ]:
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='UNK')
tokenizer.fit_on_texts(text)

fragments_indexed = tokenizer.texts_to_sequences(fragments)
targets_indexed = tokenizer.texts_to_sequences(targets)

# Convert to appropriate input and output formats.
X = np.array(fragments_indexed, dtype=np.int)
y = np.zeros((len(targets_indexed), MAX_WORDS))
for i, target_index in enumerate(targets_indexed):
    y[i, target_index] = 1

In [ ]:
print(X[0].shape)
print(y[0].shape)
print(X[0])
print(y[0])

In [ ]:
# Build and train model.
training_model = Sequential()

training_model.add(Embedding(
                        output_dim=EMBEDDING_WIDTH, 
                        input_dim=MAX_WORDS,
                        mask_zero=True, 
                        input_length=None))

training_model.add(LSTM(64, 
                        return_sequences=True,
                        dropout=0.2, 
                        recurrent_dropout=0.2))

training_model.add(LSTM(128, 
                        dropout=0.2,
                        recurrent_dropout=0.2))

training_model.add(Dense(128, activation='relu'))

training_model.add(Dense(MAX_WORDS, activation='softmax'))

In [ ]:
training_model.compile(loss='categorical_crossentropy',
                        optimizer='adam')

In [ ]:
training_model.summary()

In [ ]:
history = training_model.fit(X, y, validation_split=0.05,
                            batch_size=BATCH_SIZE,
                            epochs=EPOCHS, verbose=2,
                            shuffle=True)

In [ ]:
##Inference Model for Embedding Test

In [ ]:
inference_model = Sequential()
inference_model.add(Embedding(
                        output_dim=EMBEDDING_WIDTH, 
                        input_dim=MAX_WORDS,
                        mask_zero=True, 
                        batch_input_shape=(1, 1)))
inference_model.add(LSTM(64, 
                        return_sequences=True,
                        dropout=0.2, recurrent_dropout=0.2,
                        stateful=True))

inference_model.add(LSTM(128, dropout=0.2,
                        recurrent_dropout=0.2, stateful=True))

inference_model.add(Dense(128, activation='relu'))

inference_model.add(Dense(MAX_WORDS, activation='softmax'))

In [ ]:
weights = training_model.get_weights()
inference_model.set_weights(weights)

In [ ]:
# Provide beginning of sentence and
# predict next words in a greedy manner
first_words = ['i', 'saw']
first_words_indexed = tokenizer.texts_to_sequences(first_words)
inference_model.reset_states()
predicted_string = ''
# Feed initial words to the model.
for i, word_index in enumerate(first_words_indexed):
    x = np.zeros((1, 1), dtype=np.int)
    x[0][0] = word_index[0]
    predicted_string += first_words[i]
    predicted_string += ' '
    y_predict = inference_model.predict(x, verbose=0)[0]
# Predict PREDICT_LENGTH words.
for i in range(PREDICT_LENGTH):
    new_word_index = np.argmax(y_predict)
    word = tokenizer.sequences_to_texts([[new_word_index]])
    x[0][0] = new_word_index
    predicted_string += word[0]
    predicted_string += ' '
    y_predict = inference_model.predict(x, verbose=0)[0]
print(predicted_string)

In [ ]:
BEAM_LENGTH = 8
def BeamResetSearch(wordList):
    indexList = tokenizer.texts_to_sequences(wordList)
    inference_model.reset_states()
    predicted_string = ''
    for i, word_index in enumerate(indexList):
        x = np.zeros((1, 1), dtype=np.int)
        x[0][0] = word_index[0]
        predicted_string += wordList[i]
        predicted_string += ' '
        y_predict = inference_model.predict(x, verbose=0)[0]

    predicted_string =  predicted_string[0:-1]
    Beam = [(np.log(1.0), predicted_string)]
    newBeam = []
    print
    for k in range(BEAM_LENGTH):
        new_word_index = np.argmax(y_predict)
        word = tokenizer.sequences_to_texts([[new_word_index]])
        new_prob = Beam[0][0] + np.log(np.max(y_predict))
        new_word = predicted_string +' '+ word[0]
        newBeam.append( (new_prob, new_word) )

        y_predict[new_word_index] = 0

    return newBeam

def BeamSearch(words):
    Beams = [ (np.log(1), words) ]
    for i in range(PREDICT_LENGTH):
        all_beams = []
        for Beam in Beams:
            new_Beams = BeamResetSearch(text_to_word_sequence(Beam[1]))
            all_beams = all_beams + new_Beams
        all_beams.sort(key=lambda tup: tup[0], reverse=True)
        Beams = all_beams[0:BEAM_LENGTH]
        # print(Beams)
    return Beams


In [ ]:
words = "a body " 
predicted_beams = BeamSearch(words)
for beams in predicted_beams:
    print(beams[1])

In [ ]:
# Explore embedding similarities.
embeddings = training_model.layers[0].get_weights()[0]
lookup_words = ['the', 'saw', 'see', 'of', 'and', 'monster', 'frankenstein', 'read', 'eat']
for lookup_word in lookup_words:
    lookup_word_indexed = tokenizer.texts_to_sequences([lookup_word])
    print('words close to:', lookup_word)
    lookup_embedding = embeddings[lookup_word_indexed[0]]
    word_indices = {}
# Calculate distances.
    for i, embedding in enumerate(embeddings):
        distance = np.linalg.norm(embedding - lookup_embedding)
        word_indices[distance] = i
# Print sorted by distance.
    for distance in sorted(word_indices.keys())[:5]:
        word_index = word_indices[distance]
        word = tokenizer.sequences_to_texts([[word_index]])[0]
        print(word + ': ', distance)
    print('')